# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Module 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging than the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly by assigning it to a variable)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well-structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at a minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

# Importing `Libraries`

In [2]:
import sqlite3
import pandas as pd
import re
import requests
import json
import time
import datetime
import pymongo
import matplotlib.pyplot as plt
%matplotlib inline


# Connecting to `SQLite` database

In [3]:
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

cur.execute("""
            SELECT * FROM Matches 
            WHERE Season = 2011
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


# UPDATE the `DataFrame` USING `API` with `Custom Class` = `WeatherGetter()`

In [334]:
# You don't have to use these classes, but we recommend them as a good place to start!
class WeatherGetter(object):
    """Return an updated DataFrame when passing in a list of dates 
    Using a specific DataFrame, and API key
    Powered by Dark Sky - https://darksky.net/poweredby/
    """
    def __init__(self, dates = [], data = pd.DataFrame(), key = ''):
        """
        dates: list of dates from the dataframe - list type
        data: dataFrame object with dates - DataFrame type
        key: API key for DARK SKY API - str type
        converted_dates: auto converted dates - list type
        forecast: forecast pulled from berlin for each date - list type
        
        """
        self.dates = dates
        self.converted_dates = []
        self.forecast = []
        self.data = data
        self.key = key
        for day in self.dates:
            self.converted_dates.append(int(time.mktime(datetime.datetime.strptime(day, "%Y-%m-%d").timetuple())))
        for date in self.converted_dates:
            base_url = 'https://api.darksky.net/forecast/'
            key = self.key
            lat_long = '52.5200,13.4050'+','
            base_url = 'https://api.darksky.net/forecast/'
            url = base_url + key + "/" + lat_long + str(date)
            response = requests.get(url)
            data = response.json()
            if response.status_code == requests.codes.ok:
                icon = data['daily']['data'][0]['icon']
                self.forecast.append(icon)
            else:
                print('Connection Error')
                break 
        if len(self.forecast) == len(self.data):
            self.data['Forecast'] = self.forecast
        else:
            pass

# Instantiating `WeatherGetter()`

In [257]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)
keys = get_keys("/Users/jacobtadesse/.secret/dark_sky.json")

api_key = keys['api_key']

In [51]:
Dates = list(df.Date)
init = WeatherGetter(dates = Dates, data = df, key = api_key)
init.data

Connection Error


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Forecast
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,rain
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,partly-cloudy-day
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,partly-cloudy-day
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,partly-cloudy-day
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,partly-cloudy-day
5,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H,rain
6,1098,D1,2011,2012-02-04,Hamburg,Bayern Munich,1,1,D,partly-cloudy-day
7,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A,partly-cloudy-day
8,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A,rain
9,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H,rain


# USING a `FOR LOOP` to create a new `DataFrame` (df2)

In [106]:
teams = []
goals = []
wins = []
losses = []
draws = []
games = []
win_percent = []
percentage = []
for team in df['HomeTeam'].unique():
    ## Name
    teams.append(team)

    ## Goals
    goals_H = df['FTHG'][df['HomeTeam']==team].sum()
    goals_A = df['FTAG'][df['AwayTeam']==team].sum()
    goals.append(goals_H + goals_A)

    ## Wins
    wins_H = df[(df['HomeTeam']==team) & (df['FTR']=='H')]
    wins_A = df[(df['AwayTeam']==team) & (df['FTR']=='A')]
    wins.append(len(wins_H) + len(wins_A))

    ## Losses
    loss_H = df[(df['HomeTeam']==team) & (df['FTR']=='A')]
    loss_A = df[(df['AwayTeam']==team) & (df['FTR']=='H')]
    losses.append(len(loss_H) + len(loss_A))

    ## Draws
    draw_H = df[(df['HomeTeam']==team) & (df['FTR']=='D')]
    draw_A = df[(df['AwayTeam']==team) & (df['FTR']=='D')]
    draws.append(len(draw_H) + len(draw_A))
        
    ## Total Games
    tot_games = len(wins_H) + len(loss_H) + len(draw_H)
    games.append(tot_games)

    ## Win percentage on days where it was raining during games 
    rain_wins_H = df[(df['HomeTeam']==team) & (df['FTR']=='H') & (df['Forecast']=='rain')]
    rain_wins_A = df[(df['AwayTeam']==team) & (df['FTR']=='A') & (df['Forecast']=='rain')]
    tot_wins_in_rain = len(rain_wins_H) + len(rain_wins_A)
    percent_of_wins = tot_wins_in_rain / tot_games
    win_percent.append(round(percent_of_wins, 2))
    
    ## Rain Wins as a percent
    
    rain_wins = percent_of_wins * 100
    percent = "{}%".format(round(rain_wins,2))
    percentage.append(percent)
    
    
    
df2 = pd.DataFrame()
df2['Team'] = teams
df2['Goals'] = goals
df2['Wins'] = wins
df2['Losses'] = losses
df2['Draws'] = draws
df2['Total Games'] = games
df2['Wins in Rain'] = win_percent
df2['Rain Win Percentage'] = percentage

df2.head(10)

,Team,Goals,Wins,Losses,Draws,Total Games,Wins in Rain,Rain Win Percentage
0,Nurnberg,38,12,16,6,17,0.29,29.41%
1,Stuttgart,63,15,11,8,17,0.18,17.65%
2,Wolfsburg,47,13,16,5,17,0.24,23.53%
3,Mainz,47,9,13,12,17,0.12,11.76%
4,Freiburg,45,10,14,10,17,0.18,17.65%
5,M'gladbach,49,17,8,9,17,0.47,47.06%
6,Hamburg,35,8,14,12,17,0.12,11.76%
7,Werder Bremen,49,11,14,9,17,0.29,29.41%
8,Schalke 04,74,20,10,4,17,0.29,29.41%
9,Hannover,41,12,10,12,17,0.29,29.41%


# Saving DataFrames as `.CSV` and  `.JSON` files

### `Saving original DataFrame (df) as .CSV`

In [61]:
df.to_csv('/Users/jacobtadesse/Documents/LearnCo/mod2/s16/dsc-mod-2-lab-online-ds-pt-081219/df.csv')

### `Saving new DataFrame (df2) as .CSV`

In [108]:
df2.to_csv('/Users/jacobtadesse/Documents/LearnCo/mod2/s16/dsc-mod-2-lab-online-ds-pt-081219/df2.csv')

### `Saving new DataFrame (df2) as .JSON`

In [109]:
df2.to_json('/Users/jacobtadesse/Documents/LearnCo/mod2/s16/dsc-mod-2-lab-online-ds-pt-081219/df2.json')

# CONNECT to `MONGO DB` with `Custom Class` = `MongoHandler()` and UPLOAD  `DataFrame` 

In [333]:
class MongoHandler(object):
    """Connect to MongoDB to upload data from DataFrame
    Requires a specific database name and DataFrame
    Also allows queries when dict passed as {key:1} format
    """
    def __init__(self, db_name = '', data = pd.DataFrame(), q = {}):
        """ Initializing database name, data, and query parameters"""
        self.db_name = db_name
        self.data = data
        self.q = q

    def connect(self):
        myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
        mydb = myclient[self.db_name]
        mycollection = mydb[self.db_name]
        
    def db_names(self):
        print(myclient.list_database_names())
        
    def collection_names(self):
        print(mydb.list_collection_names())
    
    def insert(self):
        results_2 = mycollection.insert_many(self.data.to_dict('records'))
        return len(results_2.inserted_ids)
        
    def query(self):
        query = mycollection.find({}, self.q)
        for x in query:
            print(x) 
            
    def total(self):
        query = mycollection.find({})
        tot = []
        for x in query:
            tot.append(x)
        return len(tot)
        
    def delete_all(self):
        delete_all = mycollection.delete_many({})
        return delete_all.deleted_count        
        

# Instantiating `MongoHandler()`

In [326]:
upload = MongoHandler()

### Defining Database Name, Data, and Query

In [327]:
upload.db_name = '2011_team_stats'
upload.data = df2
upload.q = {'Team':1,'Wins':1, 'Losses':1, 'Total Games':1}

### Inserting Data into MongoDB using built-in method

In [328]:
upload.insert()
upload.total() # checking total

224

### Deleting all data from MongoDB using built-in method

In [330]:
upload.delete_all()
upload.insert() # checking the number inserted

56

### Querying the MongoDB using built-in method

In [331]:
upload.query()

{'_id': ObjectId('5db451ed1a2fcaabf61593e3'), 'Team': 'Nurnberg', 'Wins': 12, 'Losses': 16, 'Total Games': 17}
{'_id': ObjectId('5db451ed1a2fcaabf61593e4'), 'Team': 'Stuttgart', 'Wins': 15, 'Losses': 11, 'Total Games': 17}
{'_id': ObjectId('5db451ed1a2fcaabf61593e5'), 'Team': 'Wolfsburg', 'Wins': 13, 'Losses': 16, 'Total Games': 17}
{'_id': ObjectId('5db451ed1a2fcaabf61593e6'), 'Team': 'Mainz', 'Wins': 9, 'Losses': 13, 'Total Games': 17}
{'_id': ObjectId('5db451ed1a2fcaabf61593e7'), 'Team': 'Freiburg', 'Wins': 10, 'Losses': 14, 'Total Games': 17}
{'_id': ObjectId('5db451ed1a2fcaabf61593e8'), 'Team': "M'gladbach", 'Wins': 17, 'Losses': 8, 'Total Games': 17}
{'_id': ObjectId('5db451ed1a2fcaabf61593e9'), 'Team': 'Hamburg', 'Wins': 8, 'Losses': 14, 'Total Games': 17}
{'_id': ObjectId('5db451ed1a2fcaabf61593ea'), 'Team': 'Werder Bremen', 'Wins': 11, 'Losses': 14, 'Total Games': 17}
{'_id': ObjectId('5db451ed1a2fcaabf61593eb'), 'Team': 'Schalke 04', 'Wins': 20, 'Losses': 10, 'Total Games': 1

# USING a `FOR LOOP` to create a new `DataFrame` (df4) that includes Goals Scored for each Team's Wins, Losses, and Draws

In [252]:
teams_data = []

for team in df['HomeTeam'].unique():
    data = {}
    ## Name
    data['team'] = team
    
    ## Goals per Wins
    gphw = list(df['FTHG'][(df['HomeTeam']==team) & (df['FTR']=='H')])
    gpaw = list(df['FTAG'][(df['AwayTeam']==team) & (df['FTR']=='A')])
    data['Goals_per_Wins'] = gphw + gpaw

    ## Goals per Losses
    gphl = list(df['FTHG'][(df['HomeTeam']==team) & (df['FTR']=='A')])
    gpal = list(df['FTAG'][(df['AwayTeam']==team) & (df['FTR']=='H')])
    data['Goals_per_Losses'] = gphl + gpal

    ## Goals per Draws
    gphd = list(df['FTHG'][(df['HomeTeam']==team) & (df['FTR']=='D')])
    gpad = list(df['FTAG'][(df['AwayTeam']==team) & (df['FTR']=='D')])
    data['Goals_per_Draws'] = gphd+gpad
    
    teams_data.append(data)
           
    
df4 = pd.DataFrame(teams_data)
df4.head(10)

,Goals_per_Draws,Goals_per_Losses,Goals_per_Wins,team
0,"[1, 2, 1, 3, 0, 2]","[0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]","[2, 1, 4, 2, 1, 1, 2, 1, 3, 3, 2, 1]",Nurnberg
1,"[1, 2, 0, 4, 2, 2, 1, 1]","[1, 1, 0, 0, 0, 1, 1, 0, 0, 2, 0]","[4, 1, 3, 4, 3, 5, 4, 3, 2, 2, 2, 2, 3, 4, 2]",Stuttgart
2,"[2, 0, 0, 1, 0]","[0, 1, 2, 1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 0, 0]","[1, 3, 2, 1, 3, 2, 2, 1, 3, 4, 3, 4, 3]",Wolfsburg
3,"[0, 1, 0, 3, 0, 1, 1, 0, 1, 0, 2, 1]","[1, 2, 1, 1, 0, 0, 0, 1, 0, 2, 1, 1, 1]","[3, 4, 3, 4, 2, 2, 3, 2, 3]",Mainz
4,"[0, 2, 2, 1, 2, 0, 2, 0, 0, 1]","[1, 1, 0, 1, 1, 2, 0, 3, 2, 0, 1, 0, 0, 1]","[4, 2, 2, 3, 1, 1, 3, 2, 2, 2]",Freiburg
5,"[1, 0, 1, 2, 1, 0, 0, 0, 2]","[1, 0, 0, 1, 0, 0, 0, 0]","[3, 3, 1, 1, 3, 5, 2, 1, 4, 2, 3, 2, 1, 1, 3, ...",M'gladbach
6,"[1, 1, 1, 2, 0, 1, 1, 0, 1, 1, 1, 2]","[1, 3, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0]","[2, 1, 2, 2, 1, 1, 2, 2]",Hamburg
7,"[1, 2, 1, 1, 1, 0, 2, 1, 1]","[1, 0, 0, 2, 0, 0, 2, 0, 1, 1, 0, 1, 0, 0]","[3, 5, 2, 2, 2, 3, 2, 4, 3, 3, 2]",Werder Bremen
8,"[1, 1, 2, 1]","[0, 1, 1, 1, 1, 1, 0, 0, 0, 0]","[5, 4, 3, 2, 4, 3, 5, 3, 4, 4, 1, 3, 3, 4, 3, ...",Schalke 04
9,"[0, 1, 0, 2, 1, 1, 2, 0, 1, 1, 0, 1]","[1, 1, 0, 0, 0, 0, 1, 0, 1, 0]","[2, 2, 4, 2, 1, 4, 3, 2, 2, 2, 2, 1]",Hannover


### `Saving new DataFrame (df4) as .CSV`

In [253]:
df4.to_csv('/Users/jacobtadesse/Documents/LearnCo/mod2/s16/dsc-mod-2-lab-online-ds-pt-081219/df4.csv')

# USING a `FOR LOOP` to create `Histograms` for Each Team's Wins/Goals Scored and Losses/Goals Scored

In [338]:
team_list = list(df4['team'])
plt.figure(figsize=(32,len(team_list)*9))
k=0
for i,j in enumerate(team_list):
    
    wins = df4['Goals_per_Wins'][df4['team']== j]
    losses = df4['Goals_per_Losses'][df4['team']== j]
    
    ax1 = plt.subplot(len(team_list),2,k+1)
    ax1.hist(wins)
    ax1.set_title("{} Wins per Goals Scored in 2011".format(j),
                 fontweight='bold',
                 size=20)
    ax1.set_ylabel('# of Wins', 
                  fontsize = 20.0) 
    ax1.set_xlabel('# Goals Scored', 
                  fontsize = 20) 
    
    
    ax2 = plt.subplot(len(team_list),2,k+2)
    ax2.hist(losses)
    ax2.set_title("{} Losses per Goals Scored in 2011".format(j),
                  fontweight='bold',
                  size=20)
    ax2.set_ylabel('# of Losses', 
                  fontsize = 20.0) 
    ax2.set_xlabel('# Goals Scored', 
                  fontsize = 20) 

    k +=2

plt.show()
